<a href="https://colab.research.google.com/github/natadh/APT3025/blob/main/Intro_to_Weights%26Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install wandb


In [8]:
#log into my W&B account
import wandb
wandb.login()

True

In [9]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})

  # Mark the run as finished
  wandb.finish()

acc,▁▃▅█▆▇█▇
loss,▇█▄▄▁▂▃▂
acc,0.825
loss,0.21232


acc,▁▂▃▅▅▇▆█
loss,█▇▆▄▃▂▂▁
acc,0.88308
loss,0.10368


acc,▃▁▇▄▆███
loss,█▄▁▂▁▁▁▁
acc,0.93796
loss,0.09088


acc,▁▆▅█▇▇██
loss,█▃▄▂▁▁▁▁
acc,0.76953
loss,0.24773


acc,▃▁▄▇▇██▇
loss,█▄▃▂▂▂▂▁
acc,0.90139
loss,0.03362


In [10]:
#@title
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset,
                                         batch_size=batch_size,
                                         shuffle=True if is_train else False,
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [11]:
# Launch 5 experiments, trying different dropout rates
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(
        project="pytorch-intro",
        config={
            "epochs": 10,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": random.uniform(0.01, 0.80),
            })

    # Copy your config
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {"train/train_loss": train_loss,
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                       "train/example_ct": example_ct}

            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb
                wandb.log(metrics)

            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss,
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})

        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run
    wandb.finish()

Train Loss: 0.267, Valid Loss: 0.275996, Accuracy: 0.92
Train Loss: 0.207, Valid Loss: 0.224536, Accuracy: 0.93
Train Loss: 0.167, Valid Loss: 0.192720, Accuracy: 0.94
Train Loss: 0.075, Valid Loss: 0.171654, Accuracy: 0.95
Train Loss: 0.105, Valid Loss: 0.161492, Accuracy: 0.95
Train Loss: 0.058, Valid Loss: 0.152800, Accuracy: 0.95
Train Loss: 0.058, Valid Loss: 0.147396, Accuracy: 0.95
Train Loss: 0.027, Valid Loss: 0.141107, Accuracy: 0.95
Train Loss: 0.054, Valid Loss: 0.141807, Accuracy: 0.95
Train Loss: 0.018, Valid Loss: 0.146361, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▃▃▃▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▃▄▆▇▇███▇
val/val_loss,█▅▄▃▂▂▁▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.0183
val/val_accuracy,0.9525
val/val_loss,0.14636


Train Loss: 0.489, Valid Loss: 0.318013, Accuracy: 0.91
Train Loss: 0.390, Valid Loss: 0.255047, Accuracy: 0.93
Train Loss: 0.317, Valid Loss: 0.229121, Accuracy: 0.93
Train Loss: 0.207, Valid Loss: 0.213133, Accuracy: 0.93
Train Loss: 0.090, Valid Loss: 0.204336, Accuracy: 0.94
Train Loss: 0.248, Valid Loss: 0.198591, Accuracy: 0.94
Train Loss: 0.161, Valid Loss: 0.190779, Accuracy: 0.94
Train Loss: 0.088, Valid Loss: 0.180696, Accuracy: 0.95
Train Loss: 0.173, Valid Loss: 0.170093, Accuracy: 0.95
Train Loss: 0.134, Valid Loss: 0.163822, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▄▃▃▃▃▂▂▂▃▁▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁
val/val_accuracy,▁▄▅▅▆▇▇███
val/val_loss,█▅▄▃▃▃▂▂▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.13405
val/val_accuracy,0.949
val/val_loss,0.16382


Train Loss: 0.329, Valid Loss: 0.298246, Accuracy: 0.92
Train Loss: 0.198, Valid Loss: 0.253114, Accuracy: 0.92
Train Loss: 0.238, Valid Loss: 0.212914, Accuracy: 0.94
Train Loss: 0.245, Valid Loss: 0.199825, Accuracy: 0.93
Train Loss: 0.125, Valid Loss: 0.186464, Accuracy: 0.95
Train Loss: 0.068, Valid Loss: 0.175935, Accuracy: 0.94
Train Loss: 0.082, Valid Loss: 0.169137, Accuracy: 0.95
Train Loss: 0.141, Valid Loss: 0.166654, Accuracy: 0.95
Train Loss: 0.116, Valid Loss: 0.161931, Accuracy: 0.95
Train Loss: 0.075, Valid Loss: 0.160374, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▆▅▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▂▂▁▂▂▁▂▁▁▂
val/val_accuracy,▁▃▅▄▇▇▇▇██
val/val_loss,█▆▄▃▂▂▁▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.07498
val/val_accuracy,0.951
val/val_loss,0.16037


Train Loss: 0.267, Valid Loss: 0.310822, Accuracy: 0.91
Train Loss: 0.184, Valid Loss: 0.256812, Accuracy: 0.93
Train Loss: 0.227, Valid Loss: 0.225544, Accuracy: 0.93
Train Loss: 0.191, Valid Loss: 0.202865, Accuracy: 0.94
Train Loss: 0.141, Valid Loss: 0.186195, Accuracy: 0.94
Train Loss: 0.121, Valid Loss: 0.177667, Accuracy: 0.95
Train Loss: 0.082, Valid Loss: 0.178334, Accuracy: 0.94
Train Loss: 0.070, Valid Loss: 0.174640, Accuracy: 0.95
Train Loss: 0.128, Valid Loss: 0.160304, Accuracy: 0.95
Train Loss: 0.063, Valid Loss: 0.163030, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▃▃▃▃▃▂▂▂▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂
val/val_accuracy,▁▃▄▅▇▇▇▇██
val/val_loss,█▅▄▃▂▂▂▂▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.06324
val/val_accuracy,0.951
val/val_loss,0.16303


Train Loss: 0.313, Valid Loss: 0.289224, Accuracy: 0.92
Train Loss: 0.194, Valid Loss: 0.233442, Accuracy: 0.93
Train Loss: 0.145, Valid Loss: 0.192772, Accuracy: 0.94
Train Loss: 0.109, Valid Loss: 0.192548, Accuracy: 0.94
Train Loss: 0.031, Valid Loss: 0.175523, Accuracy: 0.95
Train Loss: 0.057, Valid Loss: 0.158656, Accuracy: 0.94
Train Loss: 0.036, Valid Loss: 0.155056, Accuracy: 0.95
Train Loss: 0.042, Valid Loss: 0.157276, Accuracy: 0.95
Train Loss: 0.018, Valid Loss: 0.152576, Accuracy: 0.96
Train Loss: 0.016, Valid Loss: 0.160023, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▄▃▃▃▃▂▂▂▂▂▁▂▂▂▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▃▆▅▇▆▇▇██
val/val_loss,█▅▃▃▂▁▁▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.01618
val/val_accuracy,0.9535
val/val_loss,0.16002


In [12]:
# Start a wandb run
wandb.init(project="pytorch-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f'Accuracy is: {accuracy}, {acc_threshold}')

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title='Low Accuracy',
            text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',
        )
        print('Alert triggered')
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 1.242, 0.3
Accuracy is: 1.845, 0.3
Accuracy is: 0.875, 0.3
Accuracy is: 1.322, 0.3
Accuracy is: 0.759, 0.3
Accuracy is: 1.556, 0.3
Accuracy is: 0.917, 0.3
Accuracy is: 1.167, 0.3
Accuracy is: 0.438, 0.3
Accuracy is: 0.707, 0.3
Accuracy is: 1.353, 0.3
Accuracy is: 1.41, 0.3
Accuracy is: 1.149, 0.3
Accuracy is: 1.055, 0.3
Accuracy is: 0.808, 0.3
Accuracy is: 0.869, 0.3
Accuracy is: 0.337, 0.3
Accuracy is: 1.308, 0.3
Accuracy is: 0.879, 0.3
Accuracy is: 1.744, 0.3
Accuracy is: 1.123, 0.3
Accuracy is: 0.64, 0.3
Accuracy is: 0.928, 0.3
Accuracy is: 1.611, 0.3
Accuracy is: 0.151, 0.3
Alert triggered


Accuracy,▆█▄▆▄▇▄▅▂▃▆▆▅▅▄▄▂▆▄█▅▃▄▇▁
Accuracy,0.151
